# LangChain 学习 - Chat Models（聊天模型）

## 本节目标
- 理解Chat Models的概念
- 学会使用OpenAI Chat Model
- 掌握不同类型的消息（System, Human, AI）
- 了解如何使用文本提示和消息提示

## 什么是Chat Model？
Chat Model是使用消息序列作为输入，返回消息作为输出的语言模型。
与传统的文本补全模型不同，Chat Model更适合对话场景。

In [1]:
# 安装必要的包
# 在终端运行：pip install langchain langchain-openai python-dotenv

# 验证安装
import langchain
print(f"LangChain version: {langchain.__version__}")

LangChain version: 1.1.3


In [2]:
# 导入环境变量
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
_project_root

sys.path.append(_project_root)

from config import config

api_key = config.OPENAI_API_KEY
base_url = config.OPENAI_BASE_URL

print(f"API Key loaded: {'✓' if api_key else '✗'}")
print(f"Base URL: {base_url}")

API Key loaded: ✓
Base URL: https://api.gptgod.online/v1


## Chat Model的三种消息类型

1. **SystemMessage** - 系统消息
    - 设置AI的行为和角色
    - 通常在对话开始时使用

2. **HumanMessage** - 人类消息
    - 用户的输入

3. **AIMessage** - AI消息
    - AI的回复
    - 在多轮对话中保存历史

In [3]:
from langchain_openai import ChatOpenAI

# 初始化ChatOpenAI模型
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=config.OPENAI_API_KEY,
    base_url=config.OPENAI_BASE_URL,
)

print("Chat Model 初始化成功")
print(f"模型名称：{model.model_name}")


Chat Model 初始化成功
模型名称：gpt-4o-mini


## Chat Model的三种核心方法

LangChain Chat Model提供三种调用方法：

### 1. invoke() - 同步调用
- 等待模型生成完整响应后一次性返回
- 适用于：简单问答、不需要实时反馈的场景
- 返回：完整的AIMessage对象

### 2. stream() - 流式输出
- 实时返回生成的内容块（chunks）
- 适用于：长文本生成、需要即时反馈的交互页面
- 返回：AIMessageChunk迭代器

### 3. batch() - 批量处理
- 并行处理多个独立请求
- 适用于：批量翻译、批量分类等场景
- 返回：AIMessage列表

In [4]:
# 方法1：invoke() - 同步调用
print("=" * 60)
print("方法1：invoke() - 等待完整响应")
print("=" * 60)

response = model.invoke("用一句话解释什么是LangChain")
print(f"类型：{type(response)}")
print(f"内容：{response.content}")

方法1：invoke() - 等待完整响应
类型：<class 'langchain_core.messages.ai.AIMessage'>
内容：LangChain是一个用于构建基于语言模型的应用程序的框架，支持多种功能模块，如数据连接、对话管理和模型调用等。


In [5]:
# 方法2：stream() - 流式输出
print("=" * 60)
print("方法2：stream() - 实时流式输出")
print("=" * 60)

print("AI回复（流式）：", end="", flush=True)

# stream()返回一个迭代器，每次yield一个chunk
for chunk in model.stream("写一首关于春天的短诗"):
    print(chunk.content, end="", flush=True)

print("\n")

方法2：stream() - 实时流式输出
AI回复（流式）：春风拂面暖阳照，  
柳枝轻摆绿意娇。  
花开遍地争相笑，  
小溪潺潺唱韵潮。  

燕子归来穿柳影，  
桃红李白映山桥。  
万物复苏生机盎，  
春天来临万象新。  



In [6]:
# stream()可以聚合成完整消息
print("=" * 60)
print("Stream chunks可以聚合成完整消息")
print("=" * 60)

full_message = None

for chunk in model.stream("用三个词描述AI"):
    # 通过累加chunk构建完整消息
    if full_message is None:
        full_message = chunk
    else:
        full_message = full_message + chunk  # 使用 + 操作符聚合
    
    print(chunk.content, end="", flush=True)

print("\n\n完整消息对象：")
print(f"类型：{type(full_message)}")
print(f"内容：{full_message.content}")
print(f"与invoke()返回的类型相同：{type(full_message).__name__ == 'AIMessage'}")

Stream chunks可以聚合成完整消息
智能、学习、自主。

完整消息对象：
类型：<class 'langchain_core.messages.ai.AIMessageChunk'>
内容：智能、学习、自主。
与invoke()返回的类型相同：False


In [7]:
# 方法3：batch() - 批量处理
print("=" * 60)
print("方法3：batch() - 并行处理多个请求")
print("=" * 60)

questions = [
    "什么是Python？",
    "什么是JavaScript？",
    "什么是Rust？",
]

responses = model.batch(questions)

for i, (question, response) in enumerate(zip(questions, responses), 1):
    print(f"\n问题 {i}: {question}")
    print(f"回答 {i}: {response.content}\n")

方法3：batch() - 并行处理多个请求

问题 1: 什么是Python？
回答 1: Python是一种高级编程语言，具有简洁的语法和强大的功能。它由Guido van Rossum于1989年首次创建，并于1991年发布了第一个公开版本。Python的设计理念强调代码的可读性和简洁性，使得程序员能够用更少的代码表达想法。

Python的主要特点包括：

1. **易读性**：Python的语法相对简单，容易学习和理解，适合初学者。

2. **丰富的库**：Python拥有大量的标准库和第三方库，可以用于各种应用，如数据分析、机器学习、网页开发、自动化脚本等。

3. **跨平台**：Python可以在不同的操作系统上运行，如Windows、macOS和Linux。

4. **多范式支持**：Python支持面向对象编程、过程式编程和函数式编程等多种编程范式。

5. **广泛应用**：Python被广泛应用于科学计算、数据分析、人工智能、网络开发、自动化测试等领域。

由于这些优点，Python在开发者社区中非常受欢迎，成为了许多初学者和专业人士的首选编程语言。


问题 2: 什么是JavaScript？
回答 2: JavaScript是一种高效的、解释型的编程语言，广泛用于网页开发。它最初由网景公司（Netscape）在1995年创建，目的是为浏览器提供动态交互功能。现在，JavaScript已经成为Web开发的核心技术之一，通常与HTML和CSS一起使用。

JavaScript的主要特点包括：

1. **动态性**：JavaScript允许在网页上动态地更新内容，响应用户输入或其他事件。
2. **事件驱动**：它能够处理用户的操作，如点击、键盘输入等，从而实现交互效果。
3. **跨平台**：JavaScript可以运行在各种设备和操作系统上，主要是在网页浏览器中执行。
4. **面向对象**：JavaScript支持面向对象的编程，可以创建和操作对象，支持继承和多态等特性。
5. **异步编程**：JavaScript支持异步编程模型，可以处理网络请求而不阻塞用户界面，提供更好的用户体验。

除了在浏览器中的应用，JavaScript还可以在服务器端使用（例如使用Node.js），以及在移动应用开发、桌面应用开发等领域。随着生态系统的

## 三种方法对比

| 方法 | 返回方式 | 用户体验 | 适用场景 | 性能 |
|------|---------|---------|---------|------|
| `invoke()` | 一次性返回 | 需要等待 | 简单问答、后台处理 | 单请求最快 |
| `stream()` | 逐块返回 | 实时反馈 | 聊天界面、长文本生成 | 首字节最快 |
| `batch()` | 批量返回 | 批量等待 | 批量翻译、分类 | 多请求最快 |

💡 **最佳实践**：
- 聊天机器人界面：使用 `stream()`
- API批量处理：使用 `batch()`
- 简单的单次请求：使用 `invoke()`

In [8]:
from langchain.messages import HumanMessage

# 使用HumanMessage包装
messages = [
    HumanMessage(content="给我讲一个关于AI的笑话")
]

response = model.invoke(messages)
print(response.content)

当然可以！这是一个关于AI的笑话：

为什么人工智能总是很冷？

因为它们总是在处理“算法”（寒冷的“气温”）！ 

希望你喜欢这个笑话！


In [9]:
from langchain.messages import SystemMessage, HumanMessage

# 使用System Message设置AI的角色和行为
messages = [
    SystemMessage(content="你是一位经验丰富的Python编程导师，擅长用简单的例子解释复杂概念。"),
    HumanMessage(content="什么是装饰器？")
]

response = model.invoke(messages)
print(response.content)

装饰器（Decorator）是Python中的一个重要特性，它允许你在不修改函数代码的情况下，为函数添加额外的功能。可以把装饰器看作是一个包装器，它可以“装饰”一个函数或方法，增强其功能。

### 基本概念

装饰器本质上是一个函数，它接受一个函数作为参数，并返回一个新的函数。这个新的函数通常会在原函数的基础上添加一些功能。

### 简单示例

下面是一个简单的装饰器示例，用于在调用函数时打印一条消息：

```python
def my_decorator(func):
    def wrapper():
        print("函数即将被调用！")
        func()  # 调用原函数
        print("函数已被调用！")
    return wrapper

@my_decorator
def say_hello():
    print("你好！")

# 调用被装饰的函数
say_hello()
```

### 输出结果

当你运行上面的代码时，输出将是：

```
函数即将被调用！
你好！
函数已被调用！
```

### 解释

1. **定义装饰器**：`my_decorator` 是一个装饰器，它接受一个函数 `func` 作为参数，并定义了一个内部函数 `wrapper`，这个 `wrapper` 函数在调用原函数 `func` 之前和之后都打印了一些信息。

2. **使用装饰器**：使用 `@my_decorator` 语法将 `say_hello` 函数装饰起来，实际上这相当于执行了 `say_hello = my_decorator(say_hello)`。

3. **调用函数**：当你调用 `say_hello()` 时，实际上调用的是 `wrapper` 函数，它在调用原始的 `say_hello` 函数之前和之后执行了额外的代码。

### 小结

装饰器非常强大，可以用于许多场景，如日志记录、权限验证、性能测试等。在Python中，装饰器是一个常见的编程模式，可以帮助你编写更简洁和模块化的代码。


In [ ]:
from langchain.messages import AIMessage

# 模拟一个对话历史
conversation = [
    SystemMessage(content="你是一个友好的助手。"),
    HumanMessage(content="我喜欢Python编程"),
    AIMessage(content="太好了！Python是一门很棒的语言。你在用Python做什么项目吗？"),
    HumanMessage(content="我在学习LangChain"),
]

response = model.invoke(conversation)
print("AI回复：")
print(response.content)

AI回复：
很不错！LangChain是一个强大的框架，用于构建与语言模型互动的应用。你对LangChain中的哪些功能或用法最感兴趣？如果你有具体问题或者想讨论某个项目，我很乐意帮忙！


In [11]:
# 多轮对话也可以使用流式输出
print("=" * 60)
print("多轮对话也可以使用流式输出")
print("=" * 60)

conversation = [
    SystemMessage(content="你是一位诗人。"),
    HumanMessage(content="请写一首关于代码的诗"),
]

print("AI回复（流式）：")
for chunk in model.stream(conversation):
    print(chunk.content, end="", flush=True)

print("\n")

多轮对话也可以使用流式输出
AI回复（流式）：
在无形的宇宙中，代码如星辰，  
逻辑的脉络编织着梦的轮廓。  
一行行字符，如诗般流淌，  
在键盘的敲击中，点燃灵感的火花。  

变量如生命，时而沉默，时而喧哗，  
循环的旋律，重复着希望与挣扎。  
条件的分岔，选择不同的道路，  
在无尽的调试中，寻找那完美的答案。  

注释是思绪的桥梁，连接过去与未来，  
函数是灵魂的片段，封存着智慧的光彩。  
每个错误都是一次成长的契机，  
在失败的碎片中，重塑新的自己。  

代码的世界，如同一幅画卷，  
色彩斑斓，逻辑交错，奇妙无比。  
创造与探索，编织着人类的梦想，  
在数字的海洋中，航行向未知的彼岸。  



## ChatOpenAI 重要参数

| 参数 | 说明 | 推荐值 |
|------|------|--------|
| `model` | 模型名称 | "gpt-4o-mini" 或 "gpt-4o" |
| `temperature` | 随机性控制 | 0-1，事实类任务用0.1-0.3，创造性任务用0.7-0.9 |
| `max_tokens` | 最大输出长度 | 根据需求，一般500-2000 |
| `streaming` | 是否默认流式输出 | True/False（控制invoke是否自动流式） |
| `api_key` | API密钥 | 从环境变量获取 |
| `base_url` | API端点 | 第三方API代理地址 |F

In [12]:
# 查看token使用情况
print("=" * 60)
print("查看token使用情况")
print("=" * 60)

response = model.invoke("什么是机器学习？用50字以内回答。")

print("AI回复：{response.content}\n")

# 检查usage_metadata
if hasattr(response, 'usage_metadata') and response.usage_metadata:
    print("Token使用情况：")
    print(f" 输入tokens: {response.usage_metadata.get('input_tokens', 'N/A')}")
    print(f" 输出toknes: {response.usage_metadata.get('output_tokens', 'N/A')}")
    print(f" 总tokens: {response.usage_metadata.get('total_tokens', 'N/A')}")
else:
    print("当前API不支持token统计")

查看token使用情况
AI回复：{response.content}

Token使用情况：
 输入tokens: 18
 输出toknes: 36
 总tokens: 54


In [13]:
# 联系：使用三种方法创建专业助手
def create_assistant_invoke(role, question):
    """使用invoke()方法"""
    messages = [
        SystemMessage(content=f"你是一位{role}"),
        HumanMessage(content=question),
    ]
    return model.invoke(messages)

def create_assistant_stream(role, question):
    """使用stream()方法"""
    messages = [
        SystemMessage(content=f"你是一位{role}"),
        HumanMessage(content=question),
    ]
    full_response = ""
    for chunk in model.stream(messages):
        print(chunk.content, end="", flush=True)
        full_response += chunk.content
    print()
    return full_response

# 测试
print("【方法1：invoke()】")
result = create_assistant_invoke("历史学家", "为什么长城如此重要？")
print(result)

print("\n【方法2：stream()】")
result = create_assistant_stream("诗人", "描述春天")

【方法1：invoke()】
content='长城是中国历史上最重要的建筑之一，其重要性体现在多个方面：\n\n1. **军事防御**：长城最初建造的目的是为了防御外敌的侵袭，尤其是来自北方游牧民族的威胁。它作为一道防线，帮助古代中国保护了其领土和人民的安全。\n\n2. **政治象征**：长城不仅是物理上的防御工事，还是中央政权力量和统一的象征。通过建造和维护长城，帝国展示了其控制和管理边疆地区的能力。\n\n3. **文化交流**：长城沿线是多个民族和文化交汇的地方。它在某种程度上促进了不同文化之间的交流与融合，同时也成为了边疆地区重要的贸易通道。\n\n4. **工程技术**：长城的建造展示了古代中国在工程技术和建筑方面的成就。它的建设历时数百年，涉及多个朝代，体现了古代劳动人民的智慧和辛勤付出。\n\n5. **旅游与文化遗产**：今天，长城被视为世界文化遗产，吸引着大量游客。它是中国悠久历史和丰富文化的象征，承载着深厚的历史记忆。\n\n6. **民族认同**：长城在中国人心中具有重要的象征意义，代表着中华民族的团结和坚韧，成为民族认同的一部分。\n\n综上所述，长城不仅在历史上具有重要的军事和政治意义，今天它更是中国文化和历史的重要象征。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 358, 'prompt_tokens': 19, 'total_tokens': 377, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': None, 'rejected_prediction_tokens': None, 'content_tokens': 358}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None, 'text_tokens': 12}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini', 's

## 本节小结

✅ 你已经学会：
1. **初始化ChatOpenAI模型**
2. **三种核心调用方法**：
   - `invoke()` - 同步调用，等待完整响应
   - `stream()` - 流式输出，实时反馈
   - `batch()` - 批量处理，并行执行
3. **三种消息类型**：SystemMessage、HumanMessage、AIMessage
4. **多轮对话**的实现方式
5. **Token使用统计**
6. **参数配置**：temperature、max_tokens等

🎯 核心概念：
- **invoke vs stream**：trade-off between simplicity and user experience
- **stream()的聚合**：chunks可以通过+操作符聚合成完整消息
- **batch()的并行**：提高多请求处理效率

💡 最佳实践：
- 聊天界面：**必须使用stream()**
- 批量任务：使用batch()提高效率
- 简单请求：invoke()即可